In [1]:
import json
from datetime import datetime
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import time as tm
from IPython.display import Image
import base64
import utils.d3
utils.d3.D3Require()

In [2]:
# Loading html utils
import utils.html
# Loading riot and dragon APIS
import utils.services
import utils.league
import utils.dragon

In [3]:
gameId = "4521402554"

In [4]:
match = utils.league.get_match(gameId)
match_timeline = utils.league.get_match_timeline(gameId)

https://euw1.api.riotgames.com/lol/match/v4/matches/4521402554?api_key=RGAPI-fcd6b25c-ff14-4b81-96ba-41ee703f626b
https://euw1.api.riotgames.com/lol/match/v4/timelines/by-match/4521402554?api_key=RGAPI-fcd6b25c-ff14-4b81-96ba-41ee703f626b


In [5]:
import lib.session
# Fetch data dragon cache
patch = '.'.join(match['gameVersion'].split('.')[:2]) + '.1' #TODO Find the closest patch
session = lib.session.Session(patch)

http://ddragon.leagueoflegends.com/cdn/10.7.1/data/en_US/champion.json
http://ddragon.leagueoflegends.com/cdn/10.7.1/data/en_US/item.json
http://ddragon.leagueoflegends.com/cdn/10.7.1/data/en_US/summoner.json


In [6]:
import lib.summary
summary = lib.summary.MatchSummary(match, session)
summary

Game ID,Name,Champion,,,,,,,,K,D,A,Gold,Minions,Lane
3,PhilBull,,,,,,,,,5,3,0,10509,197,SOLO TOP
1,God of Bug,,,,,,,,,5,1,2,9698,33,NONE JUNGLE
2,KirbySenpai,,,,,,,,,6,1,6,9609,119,SOLO MIDDLE
5,ZioNMist,,,,,,,,,4,1,9,11814,201,DUO_CARRY BOTTOM
4,B2Regret,,,,,,,,,8,5,9,9644,26,DUO_SUPPORT BOTTOM
Game ID,Name,Champion,,,,,,,,K,D,A,Gold,Minions,Lane
10,Heyo,,,,,,,,,1,4,2,7937,183,SOLO TOP
9,GUIGNOMES,,,,,,,,,4,7,2,8242,15,NONE JUNGLE
6,TRanda,,,,,,,,,1,5,3,7403,153,SOLO MIDDLE
8,ElvenBlade,,,,,,,,,2,5,4,7043,131,DUO_CARRY BOTTOM


In [ ]:
%matplotlib notebook

In [ ]:
look = summary.winners()

In [ ]:
first_frame = match_timeline['frames'][0]['participantFrames']
frame_id = {participant['participantId']: f_id for f_id, participant in first_frame.items()}

In [ ]:
xp = {}
legend = []
fig, axes = plt.subplots()
for x in range(1,11):
    linewidth = 1
    if x in look:
        linewidth = 3
    xp[f"{x}"] = [frame['participantFrames'][frame_id[x]]["xp"] for frame in match_timeline['frames']]
    line,  = axes.plot(range(0, len(xp[f"{x}"])), xp[f"{x}"], '-', linewidth = linewidth, label = f"{x}")
    legend.append(line)
    
fig.legend(handles=legend)
fig.show()

In [ ]:
gold = {}
legend = []
fig, axes = plt.subplots()
for x in range(1,11):
    linewidth = 1
    if x in look:
        linewidth = 3
    gold[f"{x}"] = [frame['participantFrames'][frame_id[x]]["totalGold"] for frame in match_timeline['frames']]
    line,  = axes.plot(range(0, len(gold[f"{x}"])), gold[f"{x}"], '-', linewidth = linewidth, label = f"{x}")
    legend.append(line)
    
fig.legend(handles=legend)
fig.show()

In [ ]:
gold = {}
legend = []
fig, axes = plt.subplots()
for x in range(1,11):
    linewidth = 1
    if x in look:
        linewidth = 3
    gold[f"{x}"] = [frame['participantFrames'][frame_id[x]]["currentGold"] for frame in match_timeline['frames']]
    line,  = axes.plot(range(0, len(gold[f"{x}"])), gold[f"{x}"], '-', linewidth = linewidth, label = f"{x}")
    legend.append(line)
    
fig.legend(handles=legend)
fig.show()

In [ ]:
gold_tot = {}
legend = []
fig, axes = plt.subplots()
for x in [True, False]:
    linewidth = 1
    label = "DEFEAT"
    if x:
        linewidth = 3
        label = "WIN"
    gold_tot[label] = [sum(
        [participant["totalGold"] for i, participant in frame['participantFrames'].items() if ((participant['participantId'] in look) == x)]
    ) for frame in match_timeline['frames']]

diff = [w - d for w,d in zip(gold_tot["WIN"], gold_tot["DEFEAT"])]
line,  = axes.plot(range(0, len(diff)), diff, '-', linewidth = linewidth, label = "GOLD DIFF")
line,  = axes.plot(range(0, len(diff)), [0 for x in diff], '-', linewidth = linewidth, label = "GOLD DIFF")
legend.append(line)
    
fig.legend(handles=legend)
fig.show()

In [ ]:
# TODO 
# - Calculate gold diff
# - Calculate power spikes
# - Find team fights
# - Evaluate damages

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

winners_x = []
winners_y = []
winners_kills_x = []
winners_kills_y = []
losers_x = []
losers_y = []
losers_kills_x = []
losers_kills_y = []

for x in range(0,len(match_timeline['frames'])-1):
    frame =  match_timeline['frames'][x]['participantFrames'].items()
    events = match_timeline['frames'][x]['events']
    winners_kills_x.append([event['position']['x'] for event in events if ((event['type'] == 'CHAMPION_KILL') and (event['killerId'] in look))])
    winners_kills_y.append([event['position']['y'] for event in events if ((event['type'] == 'CHAMPION_KILL') and (event['killerId'] in look))])
    winners_x.append([participant['position']['x'] for i, participant in frame if participant['participantId'] in look])
    winners_y.append([participant['position']['y'] for i, participant in frame if participant['participantId'] in look])

    losers_x.append([participant['position']['x'] for i, participant in frame if participant['participantId'] not in look])
    losers_y.append([participant['position']['y'] for i, participant in frame if participant['participantId'] not in look])
    losers_kills_x.append([event['position']['x'] for event in events if (event['type'] == 'CHAMPION_KILL') and (event['killerId'] not in look)])
    losers_kills_y.append([event['position']['y'] for event in events if (event['type'] == 'CHAMPION_KILL') and (event['killerId'] not in look)])

fig, axes = plt.subplots()

def f(x):
    axes.cla()
    axes.axis(xmin=0, xmax=15000, ymin=0, ymax=15000)
    #
    #winners_x = [participant['position']['x'] for i, participant in frame if participant['participantId'] in look]
    #winners_y = [participant['position']['y'] for i, participant in frame if participant['participantId'] in look]

    #losers_x = [participant['position']['x'] for i, participant in frame if participant['participantId'] not in look]
    #losers_y = [participant['position']['y'] for i, participant in frame if participant['participantId'] not in look]
    axes.plot(winners_x[x], winners_y[x], '.', markersize = 9, color='#42b3ff')
    axes.plot(winners_kills_x[x], winners_kills_y[x], 'x', markersize = 4, color='#ffb442')
    axes.plot(losers_x[x],  losers_y[x], '.', markersize = 5, color='#ffb442')
    axes.plot(losers_kills_x[x], losers_kills_y[x], 'x', markersize = 7, color='#42b3ff')

#interact(f, x = (0, (len(match_timeline['frames'])-2)))
interact(f, x=widgets.IntSlider(min=0, max=(len(match_timeline['frames'])-2), step=1, value=0));


In [7]:
summary.isWinner(accountId = )

False